In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

## Add in the numerous codes from the CRA data

In [2]:
df_matches = pd.read_csv('../joined-data/simplified_matches_uw_tenure_full.csv')

In [3]:
def rename_columns_with_prefix(df, cols, prefix):
    rename_dict = {}
    
    for col in cols:
        if col in df.columns:
            # Special case: if column is 'BN', rename to 'CRA_BN_ID'
            if col == 'BN':
                rename_dict[col] = 'CRA_BN_ID'
            else:
                # Add prefix to other columns
                rename_dict[col] = f"{prefix}_{col}".strip()
    
    # Apply the renaming
    df_renamed = df[cols].rename(columns=rename_dict)
    
    return df_renamed

Add financial data from 2011

In [4]:
df_fin11 = pd.read_csv('../cra-data/2011 CRA Charities - Financial Data(in).csv', encoding='latin')
df_fin11 = rename_columns_with_prefix(df_fin11, ['BN', '4050', '4155', '4610', '4700', '4850', '5100'], 'CRA11')
df_matches_2 = pd.merge(
    df_matches,
    df_fin11,
    on='CRA_BN_ID',
    how='left'
)

Add financial data from 2021

In [5]:
df_fin21 = pd.read_csv('../cra-data/2021 CRA Charities - Financial Data(in).csv', encoding='latin')
df_fin21 = rename_columns_with_prefix(df_fin21, ['BN', '4050', '4155', '4610', '4700', '4850', '5100'], 'CRA21')
df_matches_3 = pd.merge(
    df_matches_2,
    df_fin21,
    on='CRA_BN_ID',
    how='left'
)

/tmp/ipykernel_3423735/512825182.py:1: DtypeWarning: Columns (48,49,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fin21 = pd.read_csv('../cra-data/2021 CRA Charities - Financial Data(in).csv', encoding='latin')


Add general information from 2011

In [6]:
df_gen11 = pd.read_csv('../cra-data/2011 CRA Charities - General Information(in).csv', encoding='latin')
df_gen11 = rename_columns_with_prefix(df_gen11, ['BN', 'Program #1 Code', 'Program #1 %', 'Program #1 Desc      ', 'Program #2 Code', 'Program #2 %', 'Program #2 Desc      ', 'Program #3 Code', 'Program #3 %', 'Program #3 Desc      '], 'CRA11')
df_matches_4 = pd.merge(
    df_matches_3,
    df_gen11,
    on='CRA_BN_ID',
    how='left'
)

/tmp/ipykernel_3423735/2101371957.py:1: DtypeWarning: Columns (43,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gen11 = pd.read_csv('../cra-data/2011 CRA Charities - General Information(in).csv', encoding='latin')


Add general information from 2021

In [7]:
df_gen21 = pd.read_csv('../cra-data/2021 CRA Charities - General Information(in).csv', encoding='latin')
df_gen21 = rename_columns_with_prefix(df_gen21, ['BN', 'Program #1 Code', 'Program #1 %', 'Program #1 Desc', 'Program #2 Code', 'Program #2 %', 'Program #2 Desc', 'Program #3 Code', 'Program #3 %', 'Program #3 Desc'], 'CRA21')
df_matches_5 = pd.merge(
    df_matches_4,
    df_gen21,
    on='CRA_BN_ID',
    how='left'
)

/tmp/ipykernel_3423735/1025522038.py:1: DtypeWarning: Columns (42,45,46,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gen21 = pd.read_csv('../cra-data/2021 CRA Charities - General Information(in).csv', encoding='latin')


Save final results

In [10]:
full_percentage = 100 - ((df_matches_5.isna() | (df_matches_5.apply(lambda x: x == '' if x.dtype == 'object' else False))).sum() / len(df_matches_5) * 100).round(2)
full_percentage

CRA_BN_ID                 99.43
CRA_Organization_Name     99.60
CRA_Category              99.21
CRA_SubCategory           99.21
211_Organization_Name    100.00
211_Location_Name        100.00
211_Address               95.81
211_City                 100.00
211_Postal_Code           95.19
X_Coordinate             100.00
Y_Coordinate             100.00
Match_Method             100.00
Tenure                   100.00
UW                       100.00
Tenure_Public              1.59
CRA11_4050                 2.96
CRA11_4155                35.51
CRA11_4610                29.60
CRA11_4700                93.47
CRA11_4850                83.15
CRA11_5100                93.12
CRA21_4050                 3.53
CRA21_4155                43.63
CRA21_4610                35.11
CRA21_4700                98.24
CRA21_4850                90.07
CRA21_5100                97.44
CRA11_Program #1 Code     87.65
CRA11_Program #1 %        93.65
CRA11_Program #1 Desc     86.90
CRA11_Program #2 Code     55.18
CRA11_Pr

In [11]:
df_matches_5.to_csv('../joined-data/simplified_matches_uw_tenure_full_cra.csv', index=False)